In [1]:
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score

In [2]:
# Define paths to your datasets
healthy_data_path = 'C:/Users/rishi/Desktop/Datasets/Non COPD'
unhealthy_data_path = 'C:/Users/rishi/Desktop/Datasets/COPD'

In [3]:
# Define a function to extract features from audio files
def extract_features(audio_file_path):
    try:
        y, sr = librosa.load(audio_file_path, sr=None)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
        # Resize the feature matrix to a fixed size (e.g., 40x173)
        resized_mfccs = np.resize(mfccs, (40, 173))  # You may adjust the size as needed
        return resized_mfccs
    except Exception as e:
        print(f"Error extracting features from {audio_file_path}: {e}")
        return None

In [4]:
# Initialize lists to store spectrograms and labels
X = []
y = []


In [5]:
# Process healthy data
for file_name in os.listdir('C:/Users/rishi/Desktop/Datasets/Non COPD'):
    if file_name.endswith('.wav'):
        audio_file_path = os.path.join(healthy_data_path, file_name)
        features = extract_features(audio_file_path)
        if features is not None:
            X.append(features)
            y.append(0)  # Label for healthy patients

In [6]:
# Process unhealthy data
for file_name in os.listdir( 'C:/Users/rishi/Desktop/Datasets/COPD'):
    if file_name.endswith('.wav'):
        audio_file_path = os.path.join(unhealthy_data_path, file_name)
        features = extract_features(audio_file_path)
        if features is not None:
            X.append(features)
            y.append(1)  # Label for unhealthy patients

In [7]:
# Check shapes of elements in the lists
for i, x_elem in enumerate(X):
    print(f"Shape of element {i}: {x_elem.shape}")

Shape of element 0: (40, 173)
Shape of element 1: (40, 173)
Shape of element 2: (40, 173)
Shape of element 3: (40, 173)
Shape of element 4: (40, 173)
Shape of element 5: (40, 173)
Shape of element 6: (40, 173)
Shape of element 7: (40, 173)
Shape of element 8: (40, 173)
Shape of element 9: (40, 173)
Shape of element 10: (40, 173)
Shape of element 11: (40, 173)
Shape of element 12: (40, 173)
Shape of element 13: (40, 173)
Shape of element 14: (40, 173)
Shape of element 15: (40, 173)
Shape of element 16: (40, 173)
Shape of element 17: (40, 173)
Shape of element 18: (40, 173)
Shape of element 19: (40, 173)
Shape of element 20: (40, 173)
Shape of element 21: (40, 173)
Shape of element 22: (40, 173)
Shape of element 23: (40, 173)
Shape of element 24: (40, 173)
Shape of element 25: (40, 173)
Shape of element 26: (40, 173)
Shape of element 27: (40, 173)
Shape of element 28: (40, 173)
Shape of element 29: (40, 173)
Shape of element 30: (40, 173)
Shape of element 31: (40, 173)
Shape of element 3

In [8]:
import numpy as np

# Assuming X is a list of arrays with inconsistent shapes
# Find the maximum shape among all elements in the list
max_shape = max([x.shape for x in X])

# Initialize a new list to store resized or padded elements
X_processed = []

# Resize or pad elements to ensure uniformity in shape
for x in X:
    # Check if the shape matches the maximum shape
    if x.shape == max_shape:
        # If the shape matches, append the element as is
        X_processed.append(x)
    else:
        # If the shape does not match, resize or pad the element
        # Resize the element using bilinear interpolation
        resized_x = np.resize(x, max_shape)
        # Alternatively, pad the element with zeros to match the maximum shape
        # padded_x = np.pad(x, ((0, max_shape[0] - x.shape[0]), (0, max_shape[1] - x.shape[1])), mode='constant')
        X_processed.append(resized_x)

# Convert the processed list to a numpy array
X_processed = np.array(X_processed)


In [9]:
# Convert lists to numpy arrays
X = np.array(X)
y = np.array(y)


In [10]:
# Split the data into training, validation, and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [11]:
# Reshape the input data for compatibility with Conv2D layer
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], X_val.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)



In [12]:
# Define the CNN architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(40, 173, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Adding dropout for regularization
model.add(Dense(1, activation='sigmoid'))

C:\Users\rishi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [14]:
# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [15]:
# Train the model with validation data
history = model.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_val, y_val), callbacks=[early_stopping])


Epoch 1/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step - accuracy: 0.4853 - loss: 85.0851 - val_accuracy: 0.6149 - val_loss: 0.7765
Epoch 2/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 0.5235 - loss: 0.8040 - val_accuracy: 0.6216 - val_loss: 0.6827
Epoch 3/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.6418 - loss: 0.6675 - val_accuracy: 0.6284 - val_loss: 0.6730
Epoch 4/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.5836 - loss: 0.6974 - val_accuracy: 0.6149 - val_loss: 0.6576
Epoch 5/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.6530 - loss: 0.6712 - val_accuracy: 0.6824 - val_loss: 0.6538
Epoch 6/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.6090 - loss: 0.6630 - val_accuracy: 0.6014 - val_loss: 0.6763
Epoch 7/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.6188 - loss: 0.6643 - val_accuracy: 0.5203 - val_loss: 0.7035
Epoch 8/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.5666 - loss: 0.6544 - val_accuracy: 0.5338 - 

In [16]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.5223 - loss: 0.7036
Test accuracy: 0.532608687877655


In [18]:
def predict_patient_status_interactively(model):
    while True:
        # Prompt the user to enter the path to the audio file
        audio_file_path = input("Enter the path to the audio file (or 'quit' to exit): ")

        # Check if the user wants to quit
        if audio_file_path.lower() == 'quit':
            print("Exiting interactive prediction mode.")
            break

        try:
            # Extract features from the audio file
            features = extract_features(audio_file_path)

            # Check if features are extracted successfully
            if features is not None:
                # Reshape the features for compatibility with Conv2D layer
                features = features.reshape(1, features.shape[0], features.shape[1], 1)

                # Make prediction using the CNN model
                prediction = model.predict(features)

                # Convert prediction to human-readable label
                if prediction >= 0.5:
                    print("Predicted status: Unhealthy")
                else:
                    print("Predicted status: Healthy")
            else:
                print("Unable to extract features from the audio file.")
        except Exception as e:
            print(f"Error predicting patient status: {e}")


In [ ]:
# Example usage:
# Assuming 'model' is your trained CNN model
predict_patient_status_interactively(model)



Enter the path to the audio file (or 'quit' to exit):  Non COPD/104_1b1_Ar_sc_Litt3200.wav


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
Predicted status: Unhealthy


Enter the path to the audio file (or 'quit' to exit):  Non COPD/104_1b1_Ar_sc_Litt3200.wav


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted status: Unhealthy


Enter the path to the audio file (or 'quit' to exit):  COPD/124_1b1_Al_sc_Litt3200.wav


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted status: Unhealthy
